### DenseNet

resnet和densenet比较：

![abc]('./5_12_densenet2.svg')

在跨层连接上的主要区别：使用相加和使用连结(concat)

DenseNet的主要构建模块：稠密块(dense block)+过渡层(transition layer)

In [1]:
### 稠密块
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append('../d2lzh/')
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def conv_block(in_c, out_c):
    blk = nn.Sequential(
        nn.BatchNorm2d(in_c),
        nn.ReLU(),
        nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
    )
    return blk

In [2]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, in_c, out_c):
        super(DenseBlock, self).__init__()
        net = []
        for i in range(num_convs):
            in_ch = in_c + i * out_c
            net.append(conv_block(in_ch, out_c))
        self.net = nn.ModuleList(net)
        self.out_c = in_c + num_convs * out_c
        
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1)
        return X

In [3]:
# 例子
blk = DenseBlock(num_convs=2, in_c=3, out_c=10)
X = torch.rand(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

In [4]:
### 过渡层

In [5]:
def transition_block(in_c, out_c):
    blk = nn.Sequential(
        nn.BatchNorm2d(in_c),
        nn.ReLU(),
        nn.Conv2d(in_c, out_c, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2),
    )
    return blk

In [6]:
# 例子
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

In [7]:
### DenseNet模型

In [9]:
# 
net = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

# 
num_channels, growth_rate = 64, 32 # number_channel为当前通道数目
num_convs_in_dense_blocks = [4, 4, 4, 4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    DB = DenseBlock(num_convs, num_channels, growth_rate)
    net.add_module('DenseBlock_%d' % i, DB)
    # 获取上一个DB的输出通道，并用transition layer降低
    num_channels = DB.out_c
    if i != len(num_convs_in_dense_blocks) - 1: # 前三个DB的操作
        net.add_module('transition_block_%d' % (i), 
                       transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2
#    
net.add_module('BN', nn.BatchNorm2d(num_channels))
net.add_module('relu', nn.ReLU())
net.add_module('global_avg_pool', d2l.GlobalAvgPool2d())
net.add_module('fc', nn.Sequential(
        d2l.FlattenLayer(),
        nn.Linear(num_channels, 10)
    ))

In [11]:
# 例子（打印各block的维度）
X = torch.rand((1, 1, 96, 96))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape: ', X.shape)

0  output shape:  torch.Size([1, 64, 48, 48])
1  output shape:  torch.Size([1, 64, 48, 48])
2  output shape:  torch.Size([1, 64, 48, 48])
3  output shape:  torch.Size([1, 64, 24, 24])
DenseBlock_0  output shape:  torch.Size([1, 192, 24, 24])
transition_block_0  output shape:  torch.Size([1, 96, 12, 12])
DenseBlock_1  output shape:  torch.Size([1, 224, 12, 12])
transition_block_1  output shape:  torch.Size([1, 112, 6, 6])
DenseBlock_2  output shape:  torch.Size([1, 240, 6, 6])
transition_block_2  output shape:  torch.Size([1, 120, 3, 3])
DenseBlock_3  output shape:  torch.Size([1, 248, 3, 3])
BN  output shape:  torch.Size([1, 248, 3, 3])
relu  output shape:  torch.Size([1, 248, 3, 3])
global_avg_pool  output shape:  torch.Size([1, 248, 1, 1])
fc  output shape:  torch.Size([1, 10])


In [12]:
### 获取数据

In [14]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

In [15]:
### 训练

In [17]:
lr, num_epochs = 0.001, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, 
             batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.4549, train_acc 0.839, test acc 0.803, time 43.5 sec
epoch 2, loss 0.2727, train_acc 0.900, test acc 0.873, time 42.2 sec
epoch 3, loss 0.2344, train_acc 0.914, test acc 0.903, time 42.3 sec
epoch 4, loss 0.2094, train_acc 0.922, test acc 0.922, time 42.3 sec
epoch 5, loss 0.1925, train_acc 0.928, test acc 0.915, time 42.4 sec
epoch 6, loss 0.1770, train_acc 0.933, test acc 0.925, time 42.4 sec
epoch 7, loss 0.1618, train_acc 0.940, test acc 0.931, time 42.5 sec
epoch 8, loss 0.1540, train_acc 0.943, test acc 0.931, time 43.2 sec
epoch 9, loss 0.1432, train_acc 0.947, test acc 0.936, time 42.5 sec
epoch 10, loss 0.1345, train_acc 0.951, test acc 0.942, time 42.5 sec
